이전에 만든 결정 트리는 매개변수[ex) max_depth, min_impurity_decrease 등] 에 따라 성능이 바뀐다. 이때 배개변수를 바꾸어 가면서 테스트 세트로 지속적으로 평가하면 어떻게 될까? 결국 **모델을 테스트 세트에 점점 맞춰가게 되버린다**. 

지금까지 우리는 문제를 평가할 때 테스트 세트를 사용했다. 하지만 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 테스트 세트를 사용하지 말아야한다. 즉, **모델을 만들고 딱 한 번만 사용하는 것이 좋다**.

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 한 번 나누면 된다. 이 데이터를 **'검증 세트(validation set)'**라고 부른다.

* 보통 20 ~ 30%를 테스트 세트와 검증 세트로 떼어 놓는다. 훈련 데이터가 많다면 조금만 떼어 놓아도 문제가 없다.

훈련 세트에서 모델을 훈련하고 **검증 세트로 모델을 평가**한다. 이후 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다. 그 다음 이 매개변수를 사용해 **훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련**한다. 그리고 마지막에 테스트 세트에서 최종 점수를 평가하면 실전에 투입했을 때 테스트 세트의 점수와 비슷한 성능을 기대할 수 있다.

이전 절에서 사용한 데이터를 다시 불러와보자.

In [ ]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")

class 열을 타깃으로 하고 나머지 열은 특성 배열에 저장한다.

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

데이터를 훈련 세트와 테스트 세트로 나눈다.

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

훈련 세트인 train_input과 train_target을 훈련 세트와 검증 세트로 나눈다. 이때 test_size 매개변수를 0.2로 지정하여 train_input의 약 20%를 val_input으로 만든다.

In [ ]:
 sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

훈련 세트와 검증 세트의 크기를 확인해보자.

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


결정 트리 모델을 만들고 평가해보자.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트에 과대 적합된 것으로 보인다. 따라서 매개변수를 바꿔서 더 좋은 모델을 찾아야 한다.

문제를 해결하기 앞서 생각해볼 문제가 있다.

검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다. 하지만 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정할 것이다. 이때 **'교차 검증(cross validation)'**을 이용하면 안정적인 검증 점수를 얻고 훈련에 많은 데이터를 사용할 수 있다.

교차 검증은 **검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복**한다. 그다음 이 **점수를 평균하여 최종 검증 점수를 얻는다.**

훈련 세트를 몇 부분으로 나누냐에 따라 **k-폴드 교차 검증(k-fold cross validation)**이라고 하며 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 이용한다. 이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다. 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하므로 안정된 점수로 취급할 수 있다.

사이킷런에는 **cross_validate()**라는 교차 검증 함수가 있다. 평가할 모델 객체를 첫 번째 매개변수로 전달하고 훈련 세트 전체를 함수에 전달하면 된다.

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00793862, 0.00693655, 0.00719666, 0.00711942, 0.00706482]), 'score_time': array([0.00067377, 0.00058055, 0.00055838, 0.00056124, 0.00057817]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 모델 훈련 시간과 검증 시간을 의미한다. 각 키마다 5개의 숫자가 담겨있는데 이는 cross_validate() 함수가 기본적으로 5-폴드 교차 검증을 수행하기 때문이다. **cv 매개변수**에서 폴드 수를 바꿀 수 있다.

* fit_time, score_time은 코랩에서 리소스를 사용하는 상황에 따라 달라질 수 있으므로 컴퓨터마다 결과가 다를 수 있다.

교차 검증의 최종 점수는 test_score 키에 담긴 점수들을 평균하여 얻을 수 있다.

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다.

cross_validate() 는 **훈련 세트를 섞어 폴드를 나누지 않는다.** 우리는 앞에서 train_test_split() 함수로 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없다. 하지만 교차 검증을 할 때 훈련 세트를 섞게 될 상황이 온다면 **'분할기(splitter)'**를 지정해야한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정한다. cross_validate() 함수는 기본적으로 회귀 모델일 때는 **KFold 분할기**, 분류 모델일 때는 **StratifiedKFold**를 사용한다.

이전에 실시한 교차 검증 코드는 다음 코드와 동일하다.

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성하면 된다.

In [ ]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


이제 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자.

머신러닝 모델이 학습하는 파라미터를 '모델 파라미터'라고 한다. 반면 모델이 학습할 수 없어서 사용자가 지정해야 하는 파라미터를 '하이퍼 파라미터'라고 한다. 

우리는 이런 **하이퍼 파라미터를 튜닝해야 한다**. 우리는 다음과 같은 방식으로 파라미터를 튜닝할 수 있다.

* 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다.
* 검증 세트의 점수나 교차 검증을 통해 매개변수를 바꿔본다.

여기서 중요한 점은 **모든 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 한다**는 점이다. 결정 트리를 예로 들어보자. 결정 트리 모델에서 최적의 max_depth 값을 찾았다고 가정하자. 그 다음 이 값을 고정하고 min_samples_split을 바꿔 가며 최적의 값을 찾는다. 이렇게 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 틀린 결과가 나온다. 즉, max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 달라진다.

위와 같은 과정은 매개변수가 많아지면 복잡해진다. 우리가 직접 for문을 이용해 구현할 수 있지만 사이킷런에서는 **'그리드 서치(grid search)'**라는 도구를 제공한다.

* 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 **'AutoML'**이라고 부른다.

사이킷런의 **GridSearchCV 클래스**는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 먼저 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아보자.

GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들자.

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {"min_impurity_decrease": [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

그다음 일반 모델을 훈련하는 것처럼 fit() 메서드를 호출한다. 이 메서드를 호출하면 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행한다.

GridSearchCV의 **cv 매개변수의 기본 값은 5**이다. 따라서 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다. 즉, 25개의 모델을 훈련한다. 많은 모델을 훈련하기 때문에 GridSearchCV 클래스의 **n_jobs 매개변수**에서 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋다. 이 매개변수의 기본값은 1이고, **-1로 지정하면 시스템에 있는 모든 코어를 사용**할 수 있다.

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

교차 검증이 끝나고 최적의 파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 한다. 사이킷런의 그리드 서치는 훈련이 끝나면 **25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련**한다. 이 모델은 gs 객체의 **best_estimator_ 속성**에 저장되어 있다. 이 모델을 일반 결정 트리처럼 똑같이 사용가능한다.

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


최적의 매개변수는 **best_params_ 속성**에 저장되어있다.

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 **cv_results_ 속성**의 **'mean_test_score' 키**에 저장되어 있다.

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


가장 큰 값을 찾기 위해서는 수동으로 고르는 것 보다 **넘파이 argmax() 함수**를 사용하면 가장 큰 값의 인텍스를 추출할 수 있다. 그다음 이 인덱스를 사용해 **params 키에 저장된 매개변수를 출력**할 수 있다.

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


과정을 정리해보자.

* 탐색할 매개변수를 지정한다.
* 훈련 세트에서 그리드 서치를 수행해 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
* 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련한다.

이제 다른 매개변수도 추가해서 그리드 서치를 진행해 보자.

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

첫 번째 매개변수는 9개, 두 번째 매개변수는 15개, 세 번째 매개변수는 10개의 값을 만든다. 

따라서 이 매개변수로 수행할 교차 검증은 9 × 15 × 10 = 1,350개 이다. 기본 5-폴드 교차 검증을 수행하므로 총 6,750개의 모델이 만들어 진다. n_jobs를 -1로 지정하고 그리드 서치를 실행해 보자.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

최상의 매개변수 조합을 확인해보자.

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검즘 점수도 확인해보자

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


여기까지 그리드 서치를 사용해보았다.

하지만 앞에서 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데, 이렇게 간격을 둔 특별한 근거가 없다. 즉, 더 좁거나 넓은 간격으로 시도해 볼 필요가 있다.

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어렵다. 또 너무 많은 매개변수 조건이 있어서 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴 때 **'랜덤 서치(random search)'**를 사용할 수 있다.

랜덤 서치는 매개변수 값의 목록을 전달하는 것이 아니라 **매개변수를 샘플링할 수 있는 확률 분포 객체를 전달**한다. 싸이파이에서 2개의 확률 분포 클래스를 임포트해 보자.

In [ ]:
from scipy.stats import uniform, randint

사이파이 stats 패키지에 있는 uniform과 randint 클래스는 주어진 범위에서 고르게 값을 뽑는다. 이를 **'균등 분포에서 샘플링한다.'**라고 한다. randint는 정수, uniform은 실수를 뽑는다.

randint 객체를 만들고 0 ~ 10사이의 정숫값 10개를 샘플링해보자.

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 4, 0, 6, 4, 6, 2, 2, 9, 1])

10개밖에 되지 않기 때문에 고르게 샘플링되는 것 같지 않지만 샘플링 숫자를 늘리면 쉽게 확인할 수 있다.

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102, 115,  94,  96,  97,  88,  95, 104, 100, 109]))

이번에는 uniform으로 0 ~ 1사이의 실숫값 10개를 출력해보자.

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.04292269, 0.41956693, 0.89440568, 0.07717534, 0.7216415 ,
       0.2047173 , 0.80061002, 0.50674698, 0.85562119, 0.79445153])

랜덤 서치에 randint와 uniform 클래스 객체를 넘겨주고, 총 몇 번의 샘플링을 해서 최적의 매개변수를 찾으라고 명령할 수 있다. 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋다.

탐색할 매개변수의 딕셔너리를 만들어 보자.

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

이를 이용해 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV 클래스를 임포트해 훈련시키자.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

n_iter 매개변수에 샘플링 횟수를 입력할 수 있다. 우리는 총 100번 샘플링을 하여 교차 검증을 수행했고 최적의 매개변수 조합을 찾았다. 앞서 **그리드 서치보다 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색**할 수 있다.

최적의 매개변수 조합을 출력해보자.

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


최고의 교차 검증 점수도 확인해보자.

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_ 속성에 저장되어 있다. 이 모델로 테스트 세트의 성능을 확인해보자.

In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


테스트 세트의 점수는 검증 세트에 대한 점수보다 조금 작은 것이 일반적이다.

# 정리

핵심 포인트
* 검증 세트: 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, **테스트 세트를 사용하지 않기 위해 훈련세트에서 다시 떼어낸 데이터 세트**이다.
* 교차 검증: 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련한다.
* 그리드 서치: **하이퍼파라미터 탐색을 자동화해 주는 도구**이다. 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택한다. 마지막으로 이 매개변수 조합으로 최종 모델을 훈련한다.
* 랜덤 서치: **연속된 매개변수 값을 탐색할 때 유용**하다. 탐색 값을 샘플링할 수 있는 **확률 분포 객체를 전달**한다. 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있다.

핵심 패키지와 함수

scikit-learn
* cross_validate(): **교차 검증을 수행하는 함수**이다. 첫 번째 매개변수에 교차 검증을 수행할 **모델 객체를 절달**하고 두 번째와 세 번째 매개변수에 **특성과 타깃 데이터를 전달**한다. **scoring 매개변수**에 검증에 사용할 평가 지표를 선택할 수 있다. 기본적으로 분류 모델은 정확도를 의미하는 'accuracy', 회귀 모델은 결정계수를 의미하는 'r2'가 된다. **cv 매개변수**에 폴드 개수나 스플리터 객체를 지정할 수 있다. 기본값은 5이다. 회귀일 때는 KFold 클래스, 분류는 StratifiedKFold 클래스를 사용한다. **n_jobs 매개변수**는 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정한다. **return_train_score 매개변수**를 True로 지정하면 훈련 세트의 점수도 반환한다. 기본값은 False이다.

* GridSearchCV: **교차 검증으로 하이퍼파라미터 탐색을 수행**한다. 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련한다. 첫 번째 매개변수로 그리드 서치를 수행할 **모델 객체를 전달**하고 두 번째 매개변수에는 탐색할 모델의 **매개변수와 값을 전달**한다. scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일하다.

* RandomizedSearchCV: **교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행**한다. 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련한다. 첫 번째 매개변수로 **그리드 서치를 수행할 모델 객체를 전달**하고 두 번째 매개변수에는 탐색할 모델의 **매개변수와 확률 분포 객체를 전달**한다. scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일하다.